In [1]:
from main_2 import generate_script_gemini, extract_list_from_text

c:\Users\GLB-BLR-307\Documents\VideoGeneration\venv_vid_gen_312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
secs = 50
secs_for_each_clip = 5
number_of_clips_to_merge = secs // secs_for_each_clip

In [3]:
number_of_clips_to_merge

10

In [4]:
gemini_response, raw = generate_script_gemini(instruction="Create an add for employee monitoring system and its ease of integration with the workspace", brand_name="EMP Monitor", tag_line="Time matters", sale=True,time_spans=[str(secs_for_each_clip)+" secs" for i in range(number_of_clips_to_merge)])


Prepare a video ad script that will appear as subtitles over a merged video containing different time spans, as provided:  
['5 secs', '5 secs', '5 secs', '5 secs', '5 secs', '5 secs', '5 secs', '5 secs', '5 secs', '5 secs']  

**INSTRUCTION**: "General purpose Create an add for employee monitoring system and its ease of integration with the workspace"  
Other_information: Brand name is "Emp monitor", Tag line of brand is "Time matters", Our sale is also live.  

**Output Pattern**:  
A list of pairs in the format:  
`[[<short_sentence_with_keywords>, <longer_sentence>], [<short_sentence_with_keywords>, <longer_sentence>], ...]`  

- **Short Sentence**: 5 words or fewer, containing visually concrete and specific keywords.  
    - Examples: "Cheetah running", "Rainy street", "Sunset beach".  
    - Avoid vague phrases like "beautiful view" or "emotional moment".  
- **Longer Sentence**: Descriptive context (within 50 words) that complements the short sentence.  

**HINTS**:  
- Use shor

In [5]:
raw

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "```\n<BEGIN> [['Monitor employee activity', 'EmpMonitor: Seamlessly track employee performance.'], ['Easy integration process', 'Effortless setup with your existing workspace.'], ['Boost team productivity', 'Improve efficiency and achieve better results.'], ['Real-time data insights', 'Gain valuable insights into team performance.'], ['Data-driven decisions', 'Make informed decisions based on accurate data.'], ['Sale now live!', 'Get EmpMonitor now at a special price!'], ['Time matters', 'EmpMonitor helps you make the most of your time.'], ['Secure employee data', 'Safe and reliable platform to monitor your team.'], ['Improved workplace efficiency', 'Streamline operations with EmpMonitor.'], ['Increase workplace productivity', 'Maximize productivity with Emp

In [6]:
gemini_response

[['Monitor employee activity',
  'EmpMonitor: Seamlessly track employee performance.'],
 ['Easy integration process',
  'Effortless setup with your existing workspace.'],
 ['Boost team productivity', 'Improve efficiency and achieve better results.'],
 ['Real-time data insights', 'Gain valuable insights into team performance.'],
 ['Data-driven decisions', 'Make informed decisions based on accurate data.'],
 ['Sale now live!', 'Get EmpMonitor now at a special price!'],
 ['Time matters', 'EmpMonitor helps you make the most of your time.'],
 ['Secure employee data', 'Safe and reliable platform to monitor your team.'],
 ['Improved workplace efficiency', 'Streamline operations with EmpMonitor.'],
 ['Increase workplace productivity',
  "Maximize productivity with EmpMonitor's simple interface."]]

In [7]:
d ="```\n<BEGIN> [['Monitor employee productivity', 'EmpMonitor: Boost team efficiency with easy integration.'], ['Real-time data insights', 'See your team\u2019s work in action, increase productivity.'], ['Effortless workspace setup', 'Integrate EmpMonitor seamlessly into your existing workflow.'], ['Sales event live now', 'Don\u2019t miss our limited-time offer! Time matters.'], ['Improved team management', 'EmpMonitor provides real-time data for better team management.'], ['Secure employee data', 'EmpMonitor ensures your employee data is safe and secure.'], ['Streamlined workflow', 'See how EmpMonitor simplifies your workflow and saves time.'], ['Increase team output', 'Maximize your team\u2019s potential with EmpMonitor\u2019s insights.'], ['Simple user interface', 'Easy to use, even for those new to employee monitoring.'], ['Time matters, act now', 'Our limited-time sale on EmpMonitor is ending soon!']] <END>\n```\n"


In [8]:
import json
s = (d[d.find("<BEGIN>")+7:d.find("<END>")]).strip()

In [9]:
s

"[['Monitor employee productivity', 'EmpMonitor: Boost team efficiency with easy integration.'], ['Real-time data insights', 'See your team’s work in action, increase productivity.'], ['Effortless workspace setup', 'Integrate EmpMonitor seamlessly into your existing workflow.'], ['Sales event live now', 'Don’t miss our limited-time offer! Time matters.'], ['Improved team management', 'EmpMonitor provides real-time data for better team management.'], ['Secure employee data', 'EmpMonitor ensures your employee data is safe and secure.'], ['Streamlined workflow', 'See how EmpMonitor simplifies your workflow and saves time.'], ['Increase team output', 'Maximize your team’s potential with EmpMonitor’s insights.'], ['Simple user interface', 'Easy to use, even for those new to employee monitoring.'], ['Time matters, act now', 'Our limited-time sale on EmpMonitor is ending soon!']]"

In [10]:
eval(s.replace(":", ""))

[['Monitor employee productivity',
  'EmpMonitor Boost team efficiency with easy integration.'],
 ['Real-time data insights',
  'See your team’s work in action, increase productivity.'],
 ['Effortless workspace setup',
  'Integrate EmpMonitor seamlessly into your existing workflow.'],
 ['Sales event live now', 'Don’t miss our limited-time offer! Time matters.'],
 ['Improved team management',
  'EmpMonitor provides real-time data for better team management.'],
 ['Secure employee data',
  'EmpMonitor ensures your employee data is safe and secure.'],
 ['Streamlined workflow',
  'See how EmpMonitor simplifies your workflow and saves time.'],
 ['Increase team output',
  'Maximize your team’s potential with EmpMonitor’s insights.'],
 ['Simple user interface',
  'Easy to use, even for those new to employee monitoring.'],
 ['Time matters, act now',
  'Our limited-time sale on EmpMonitor is ending soon!']]

In [11]:
eval(raw.parts[0].text.strip("```python\n").strip("```"))

SyntaxError: invalid syntax (<string>, line 1)

In [ ]:
import textwrap
textwrap.wrap("hi hello welcom namastey sat sri akal vadakkam kemchho ", 25)

['hi hello welcom namastey', 'sat sri akal vadakkam', 'kemchho']

In [ ]:
# <!-- Sure! Here's the prompt formatted as a Python string: -->

prompt = '''Prepare a video ad script that will appear as subtitles over a merged video containing different time spans, as provided:  
{str(time_spans)}  

**INSTRUCTION**: "General purpose {instruction}"  
{other_information}  

**Output Pattern**:  
A list of pairs in the format:  
`[[<short_sentence_with_keywords>, <longer_sentence>], [<short_sentence_with_keywords>, <longer_sentence>], ...]`  

- **Short Sentence**: 5 words or fewer, containing visually concrete and specific keywords.  
    - Examples: "Cheetah running", "Rainy street", "Sunset beach".  
    - Avoid vague phrases like "beautiful view" or "emotional moment".  
- **Longer Sentence**: Descriptive context (within 50 words) that complements the short sentence.  

**HINTS**:  
- Use short sentences as **searchable visual keywords** for matching relevant background visuals.  
- If a sentence lacks clarity, combine it with context from nearby sentences to infer the correct visual keywords.  
- Use **English only** in the responses.  

**EXPECTED RESPONSE**:  
```
<BEGIN> [[<short_sentence_1>, <longer_sentence_1>], [<short_sentence_2>, <longer_sentence_2>], ...] <END>
```

**Response Guidelines**:  
1. Strictly return a **nested list** formatted as specified above.  
2. **Do not include** additional items, notes, symbols, or encodings (e.g., no `\u201c` or explanations).  
3. Ensure:  
    - Short sentences are visually specific and contain 5 words or fewer.  
    - Longer sentences are descriptive, stay within 50 words, and relate to the short sentence.  

**Example Response**:  
```
<BEGIN> [['Snowy mountain peak', 'A majestic view of snow-covered peaks under the clear blue sky.'], ['City skyline at dusk', 'The city lights begin to glow as the sun sets over the horizon.'], ['Forest trail morning sunlight', 'Sunlight filters through tall trees, illuminating a quiet forest path.']] <END>
```
'''


# <!-- This Python string format allows you to easily integrate the prompt into your code. Let me know if you need any adjustments! -->